<a href="https://colab.research.google.com/github/Yshen-11/DUKE_XAI/blob/main/Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import joblib

# ===============================
# 1. 加载模型和标准化器
# ===============================
# 加载预训练的随机森林模型和标准化器
model = joblib.load("best_rf_model.pkl")  # 替换为实际模型路径
scaler = joblib.load("scaler.pkl")       # 替换为实际标准化器路径

# 定义特征名称
features = ['Yrs', 'G', 'PPG', 'RPG', 'APG', 'WS', 'BPM', 'MPG',
            'Efficiency', 'ScoringEfficiency', 'DefensiveImpact',
            'ClutchAbility', 'OffDefBalance', 'Potential']

# ===============================
# 2. Streamlit 页面布局
# ===============================
st.title("NBA 选秀预测系统")
st.write("请输入球员的相关数据，系统将预测其选秀顺位，并提供预测解释。")

# ===============================
# ===============================
# 3. 用户输入表单
# ===============================
# 创建用户输入区域
st.sidebar.header("输入球员特征")
input_data = {}

for feature in features:
    # 根据特征类型设置适当的输入方式
    if feature == 'Yrs':  # 整数类型的特征
        input_data[feature] = st.sidebar.number_input(
            f"请输入 {feature}:", value=1, step=1
        )
    else:  # 浮点数类型的特征
        input_data[feature] = st.sidebar.number_input(
            f"请输入 {feature}:", value=0.0, step=0.1
        )

# 转换为 pandas 数据框
input_df = pd.DataFrame([input_data])


# ===============================
# 4. 数据标准化和预测
# ===============================
# 预测按钮
if st.sidebar.button("预测"):
    # 标准化数据
    input_scaled = scaler.transform(input_df)

    # 模型预测
    prediction = model.predict(input_scaled)

    # 显示预测结果
    st.subheader("预测结果")
    st.write(f"预测的选秀顺位: **{int(prediction[0])}**")

    # ===============================
    # 5. SHAP 特征解释
    # ===============================
    # 生成 SHAP 值
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(input_scaled)

    # 显示特征重要性图
    st.subheader("特征重要性解释")
    shap.initjs()
    shap.force_plot(
        explainer.expected_value[0],
        shap_values[0],
        input_df,
        matplotlib=True
    )
    plt.savefig("shap_plot.png")
    st.image("shap_plot.png")

    # 全局解释
    st.write("下方为单次预测的特征贡献图：")
    shap.summary_plot(shap_values, input_df, feature_names=features)
    st.pyplot(plt)